In [55]:
import os
import torch
import numpy as np

print("Using kernel:", os.environ['CONDA_DEFAULT_ENV'])

! which python
# ! jupyter kernelspec list --json

# Should show a GPU being used
# ! nvidia-smi

# Creates a cache for pretrained models/datasets in the scratch dir
# !mkdir /fs/nexus-scratch/yzhang42/.hf_cache

CACHE_DIR = "/fs/nexus-scratch/yzhang42/.hf_cache"
%set_env HF_HOME=/fs/nexus-scratch/yzhang42/.hf_cache/
%set_env TRANSFORMERS_CACHE=/fs/nexus-scratch/yzhang42/.hf_cache/transformers
%set_env HF_DATASETS_CACHE=/fs/nexus-scratch/yzhang42/.hf_cache/datasets
# "/fs/nexus-scratch/yzhang42/.hf_cache/datasets/wiki_dpr/psgs_w100.multiset.exact/0.0.0/74d4bff38a7c18a9498fafef864a8ba7129e27cb8d71b22f5e14d84cb17edd54"
print("Storing datasets in:", os.environ['HF_DATASETS_CACHE'])

device = torch.device('cuda')
print(f"Using device: {device}")

Using kernel: base
/Users/zhaoyujian/miniconda3/bin/python
env: HF_HOME=/fs/nexus-scratch/yzhang42/.hf_cache/
env: TRANSFORMERS_CACHE=/fs/nexus-scratch/yzhang42/.hf_cache/transformers
env: HF_DATASETS_CACHE=/fs/nexus-scratch/yzhang42/.hf_cache/datasets
Storing datasets in: /fs/nexus-scratch/yzhang42/.hf_cache/datasets
Using device: cuda


In [56]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from models import Model_Rational_Label
tokenizer = AutoTokenizer.from_pretrained("/Users/zhaoyujian/bert-base-uncased-hatexplain", local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained("/Users/zhaoyujian/bert-base-uncased-hatexplain", local_files_only=True)
# tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
# model = Model_Rational_Label.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")


In [79]:
test = "how are you"
def predict(text):
    text = str(text)
    inputs = tokenizer(str(text), return_tensors="pt")
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)\
            .detach().numpy().astype(float)[0].round(decimals=2)
    return probs

probs = predict(test)
probs

array([0.07, 0.65, 0.29])

In [82]:
# Import the necessaries libraries
import plotly.graph_objs as go
# Set notebook mode to work in offline

def graphProbs(probs):
    labels = ['Hate-Speech', 'Normal', 'Offensive']
    colors = ['indianred', 'darkgreen', 'gold']
    fig = go.Figure(data=[go.Pie(labels=labels,
                                 values=probs,
                                 hole=0.6,
#                                  color_discrete_map=color_discrete_map
                                )]
                    )
    fig.update_traces(
        title="Classification Probabilities",
        hoverinfo='label+percent',
        textinfo='label+percent',
        textfont_size=20,
        marker=dict(colors=colors, line=dict(color='DarkSlateGrey', width=2)))
    return fig

In [83]:
graphProbs(probs).show()

In [60]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import transformers
import shap
import plotly.express as px

pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)
explainer = shap.Explainer(pred)
def _predict(text):
    [results] = pred(text)
    labels = [result['label'] for result in results]
    probs = [result['score'] for result in results]

    return {'labels': labels, 'probs': probs}

In [71]:
pred("how are you")

[[{'label': 'normal', 'score': 0.6459826827049255},
  {'label': 'offensive', 'score': 0.28649407625198364},
  {'label': 'hate speech', 'score': 0.06752332299947739}]]

In [61]:
def updateSHAP(input_value):
    res = _predict(input_value)
    labels, probs = res['labels'], res['probs']
    label2color = {'hate speech':'indianred', 'normal':'darkgreen', 'offensive':'gold'}
    # colors = [label2color[label] for label in labels]

    if input_value.isspace() or input_value == "" or input_value == "What's happening?":
        probs = [int(label == 'normal') for label in labels]

    shap_values = explainer([input_value])
    temp_len = len(shap_values.data[0])

    toks = list(shap_values[0,1:temp_len-1,:].data)
    data = {"Token": [], 'value': [], 'label': []}

    for i, label in enumerate(labels):
        data['Token'] += toks
        data['value'] += list(shap_values[0,1:temp_len-1,i].values)
        data['label'] += [label]*len(toks)

    df = pd.DataFrame(data=data)
    print(df)

    SHAP_graph = px.bar(df, x='Token', y='value',
                        color='label', color_discrete_map=label2color,
                        barmode='group')
    SHAP_graph.show()


In [62]:
updateSHAP("How are you")

  Token     value        label
0  How  -0.130139       normal
1  are  -0.052764       normal
2   you -0.029131       normal
3  How   0.218957    offensive
4  are   0.063927    offensive
5   you  0.058770    offensive
6  How  -0.088818  hate speech
7  are  -0.011162  hate speech
8   you -0.029638  hate speech


In [90]:
from lime.lime_text import LimeTextExplainer
import torch.nn.functional as F

explainer = LimeTextExplainer(class_names=['Hate-Speech', 'Normal', 'Offensive'])

def predict_prob(text):
    outputs = model(**tokenizer(text, return_tensors="pt", padding=True))
    probas = F.softmax(outputs.logits).detach().numpy()
    return probas

exp = explainer.explain_instance("How are you", predict_prob)

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [92]:
exp.save_to_file("LIME.html")